In [1]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
train_transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]
)

In [4]:
trainset = datasets.CIFAR10('./data', train = True, transform=train_transform, download = True)
trainloader = DataLoader(trainset,batch_size = 64, shuffle = True, num_workers = 2)

testset = datasets.CIFAR10('./data', train = False, transform = test_transform, download=True)
testloader = DataLoader(testset, batch_size=64, shuffle = False, num_workers = 2)


Files already downloaded and verified
Files already downloaded and verified


The VGG-net concept is the following:
    1. It can be a deeper arhitecture
    2. Every important block (fully connected layers not included) have 2 convolutional layers and only 1 pooling layer
    3. The first convolution layer in a block is classic(output channels > input channels) but the 2nd one has the same amount of channels in the input and output
        - Why is this important? Well, having two convolutional layers with a kernel size = 3 (3x3) but the same amount of features as implementing one convolutional layer means that the actual output is similar to a convolutional layer with a kernel size = 5 and with more understanding of the features (having more connections in this case means more "intelligence")

In [14]:
class vgg_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2),
            nn.Dropout(0.2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.4)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*4*4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.fc(x)
        return x


In [15]:
model = vgg_model().to(device)
model

vgg_model(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=20, gamma = 0.5)

In [17]:
epochs = 50

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(trainloader):.4f}")

Epoch 1/50 | Loss: 1.6958
Epoch 2/50 | Loss: 1.3541
Epoch 3/50 | Loss: 1.1904
Epoch 4/50 | Loss: 1.0727
Epoch 5/50 | Loss: 0.9895
Epoch 6/50 | Loss: 0.9235
Epoch 7/50 | Loss: 0.8692
Epoch 8/50 | Loss: 0.8302
Epoch 9/50 | Loss: 0.7871
Epoch 10/50 | Loss: 0.7636
Epoch 11/50 | Loss: 0.7361
Epoch 12/50 | Loss: 0.7144
Epoch 13/50 | Loss: 0.6851
Epoch 14/50 | Loss: 0.6625
Epoch 15/50 | Loss: 0.6520
Epoch 16/50 | Loss: 0.6253
Epoch 17/50 | Loss: 0.6106
Epoch 18/50 | Loss: 0.5972
Epoch 19/50 | Loss: 0.5836
Epoch 20/50 | Loss: 0.5748
Epoch 21/50 | Loss: 0.5318
Epoch 22/50 | Loss: 0.5237
Epoch 23/50 | Loss: 0.5199
Epoch 24/50 | Loss: 0.5146
Epoch 25/50 | Loss: 0.5033
Epoch 26/50 | Loss: 0.4954
Epoch 27/50 | Loss: 0.4878
Epoch 28/50 | Loss: 0.4863
Epoch 29/50 | Loss: 0.4846
Epoch 30/50 | Loss: 0.4732
Epoch 31/50 | Loss: 0.4750
Epoch 32/50 | Loss: 0.4634
Epoch 33/50 | Loss: 0.4616
Epoch 34/50 | Loss: 0.4537
Epoch 35/50 | Loss: 0.4482
Epoch 36/50 | Loss: 0.4442
Epoch 37/50 | Loss: 0.4432
Epoch 38/5

In [18]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
acc = 100 * correct/total
print(f"Test accuracy: {acc}%")

Test accuracy: 87.81%


The accuracy of this custom made model is way higher than the ones that are not following a state-of-the-art arhitecture.
If we want a better outcome, we have to use a model that has a ResNet-like arhitecture or use transfer learning (which in this case is not really a learning experience).

In [19]:
path = './vgg_model.pth'
torch.save(model.state_dict(), path)